In [1]:
#Importation des libraries
import pandas as pd
import numpy as np
from datetime import timedelta

In [53]:
pic_journal = pd.read_csv("pic-journalier-consommation-brute.csv", sep=";")

In [54]:
pic_journal=pic_journal.sort_values(['Date'])

In [55]:

# Convertir la colonne 'Date' en datetime si ce n'est pas déjà fait
pic_journal['Date'] = pd.to_datetime(pic_journal['Date'])


In [56]:
# Étendre chaque jour à 24 heures
hourly_data = []
for _, row in pic_journal.iterrows():
    for hour in range(24):
        # Ajouter une ligne par heure
        hourly_data.append({
            'DELIVERY_START': row['Date'] + timedelta(hours=hour),  # Ajoute l'heure au jour de base
            'Consommation (MW)': row['Pic journalier consommation (MW)'],  # Utilise la même valeur
            'Température moyenne (°C)': row['Température moyenne (°C)'],    # Valeur constante
            'Température référence (°C)': row['Température référence (°C)'] # Valeur constante
        })

# Créer un DataFrame horaire
hourly_df = pd.DataFrame(hourly_data)

In [57]:
# Définir les dates de début et de fin pour le filtre
start_date = '2022-01-01 02:00:00'
end_date = '2023-03-29 23:00:00'

# Filtrer les données entre les dates de début et de fin
filtered_df = hourly_df[(hourly_df['DELIVERY_START'] >= start_date) & (hourly_df['DELIVERY_START'] <= end_date)]

# Trier par date
filtered_df = filtered_df.sort_values(by='DELIVERY_START').reset_index(drop=True)

# Afficher les premières lignes pour vérifier
print(filtered_df.head())
print(filtered_df.tail())

       DELIVERY_START  Consommation (MW)  Température moyenne (°C)  \
0 2022-01-01 02:00:00            55828.0                       9.7   
1 2022-01-01 03:00:00            55828.0                       9.7   
2 2022-01-01 04:00:00            55828.0                       9.7   
3 2022-01-01 05:00:00            55828.0                       9.7   
4 2022-01-01 06:00:00            55828.0                       9.7   

   Température référence (°C)  
0                         4.6  
1                         4.6  
2                         4.6  
3                         4.6  
4                         4.6  
           DELIVERY_START  Consommation (MW)  Température moyenne (°C)  \
10865 2023-03-29 19:00:00            58250.0                      14.0   
10866 2023-03-29 20:00:00            58250.0                      14.0   
10867 2023-03-29 21:00:00            58250.0                      14.0   
10868 2023-03-29 22:00:00            58250.0                      14.0   
10869 2023-03-29 

# courbes-de-production-mensuelles-eolien-solaire-complement-de-remuneration

In [58]:
produc_eo = pd.read_csv("courbes-de-production-mensuelles-eolien-solaire-complement-de-remuneration.csv", sep=";")

In [59]:
produc_eo=produc_eo.sort_values(by=['Date', 'Heure']).reset_index(drop=True)
# 1. Convertir la colonne 'Date' en datetime (format année-mois-jour)
produc_eo['Date'] = pd.to_datetime(produc_eo['Date'], format='%Y-%m-%d')

# 2. Convertir la colonne 'Heure' en timedelta (en ajoutant les secondes :00)
produc_eo['Heure'] = pd.to_timedelta(produc_eo['Heure'] + ":00")

# 3. Ajouter la colonne 'Heure' à la colonne 'Date' pour créer une nouvelle colonne 'DateTime'
produc_eo['DELIVERY_START'] = produc_eo['Date'] + produc_eo['Heure']
# 4. (Optionnel) Supprimer les colonnes 'Date' et 'Heure' si vous n'en avez plus besoin
produc_eo = produc_eo.drop(columns=['Date', 'Heure'])

# Afficher les premières lignes pour vérifier
print(produc_eo.head())

   prod_eolienne_MWh  prod_solaire_MWh      DELIVERY_START
0             3153.0               2.0 2022-01-01 00:00:00
1             3219.0               1.0 2022-01-01 01:00:00
2             3187.0               1.0 2022-01-01 02:00:00
3             3390.0               1.0 2022-01-01 03:00:00
4             3558.0               1.0 2022-01-01 04:00:00


In [60]:
# Définir les dates de début et de fin pour le filtre
start_date = '2022-01-01 02:00:00'
end_date = '2023-03-29 23:00:00'

# Filtrer les données entre les dates de début et de fin
filtered_produc = produc_eo[(produc_eo['DELIVERY_START'] >= start_date) & (produc_eo['DELIVERY_START'] <= end_date)]

# Trier par date
filtered_produc = filtered_produc.sort_values(by='DELIVERY_START').reset_index(drop=True)

# Afficher les premières lignes pour vérifier
#print(filtered_produc.head())
print(filtered_produc.tail())

       prod_eolienne_MWh  prod_solaire_MWh      DELIVERY_START
10864             8759.0             325.0 2023-03-29 19:00:00
10865            10026.0               9.0 2023-03-29 20:00:00
10866            11140.0               3.0 2023-03-29 21:00:00
10867            11813.0               2.0 2023-03-29 22:00:00
10868            12067.0               2.0 2023-03-29 23:00:00


In [61]:
data_complet = pd.merge(filtered_produc, filtered_df , on="DELIVERY_START")

In [62]:
Xty = pd.read_csv("X_train.csv", sep=",")

In [63]:
# Convertir en datetime avec utc=True pour unifier les fuseaux horaires
Xty['DELIVERY_START'] = pd.to_datetime(Xty['DELIVERY_START'], errors='coerce', utc=True)

# Convertir en heure locale, puis supprimer le fuseau horaire pour garder l'heure inchangée
Xty['DELIVERY_START'] = Xty['DELIVERY_START'].dt.tz_convert('Europe/Paris').dt.tz_localize(None)

In [64]:
data_info = pd.merge(Xty, data_complet , on="DELIVERY_START")

In [67]:
data_info.shape

(10605, 15)

In [68]:
#Save the results to a CSV file
data_info.to_csv('X_train_add.csv', index=False)

# Zone test

In [69]:
# Définir les dates de début et de fin pour le filtre
start_date = '2023-04-02 00:00'
end_date = '2023-10-24 23:00'

# Filtrer les données entre les dates de début et de fin
filtered_test_pic = hourly_df[(hourly_df['DELIVERY_START'] >= start_date) & (hourly_df['DELIVERY_START'] <= end_date)]

# Trier par date
filtered_test_pic = filtered_test_pic.sort_values(by='DELIVERY_START').reset_index(drop=True)

# Afficher les premières lignes pour vérifier
print(filtered_test_pic.head())
print(filtered_test_pic.tail())

       DELIVERY_START  Consommation (MW)  Température moyenne (°C)  \
0 2023-04-02 00:00:00            53630.0                       9.2   
1 2023-04-02 01:00:00            53630.0                       9.2   
2 2023-04-02 02:00:00            53630.0                       9.2   
3 2023-04-02 03:00:00            53630.0                       9.2   
4 2023-04-02 04:00:00            53630.0                       9.2   

   Température référence (°C)  
0                        10.2  
1                        10.2  
2                        10.2  
3                        10.2  
4                        10.2  
          DELIVERY_START  Consommation (MW)  Température moyenne (°C)  \
4939 2023-10-24 19:00:00            54269.0                      13.4   
4940 2023-10-24 20:00:00            54269.0                      13.4   
4941 2023-10-24 21:00:00            54269.0                      13.4   
4942 2023-10-24 22:00:00            54269.0                      13.4   
4943 2023-10-24 23:00:

In [70]:
# Définir les dates de début et de fin pour le filtre
start_date = '2023-04-02 00:00'
end_date = '2023-10-24 23:00'
# Filtrer les données entre les dates de début et de fin
filtered_produc_test = produc_eo[(produc_eo['DELIVERY_START'] >= start_date) & (produc_eo['DELIVERY_START'] <= end_date)]

# Trier par date
filtered_produc_test = filtered_produc_test.sort_values(by='DELIVERY_START').reset_index(drop=True)

# Afficher les premières lignes pour vérifier
print(filtered_produc_test.head())
print(filtered_produc_test.tail())

   prod_eolienne_MWh  prod_solaire_MWh      DELIVERY_START
0             8397.0               0.0 2023-04-02 00:00:00
1             8789.0               0.0 2023-04-02 01:00:00
2             8919.0               0.0 2023-04-02 02:00:00
3             8956.0               0.0 2023-04-02 03:00:00
4             8862.0               0.0 2023-04-02 04:00:00
      prod_eolienne_MWh  prod_solaire_MWh      DELIVERY_START
4939             4183.0               2.0 2023-10-24 19:00:00
4940             5058.0               1.0 2023-10-24 20:00:00
4941             5848.0               1.0 2023-10-24 21:00:00
4942             6398.0               1.0 2023-10-24 22:00:00
4943             7028.0               1.0 2023-10-24 23:00:00


In [71]:
data_complet_test = pd.merge(filtered_test_pic, filtered_produc_test , on="DELIVERY_START")

In [72]:
X_test = pd.read_csv("X_test.csv", sep=",")

In [73]:
# Convertir en datetime avec utc=True pour unifier les fuseaux horaires
X_test['DELIVERY_START'] = pd.to_datetime(X_test['DELIVERY_START'], errors='coerce', utc=True)

# Convertir en heure locale, puis supprimer le fuseau horaire pour garder l'heure inchangée
X_test['DELIVERY_START'] = X_test['DELIVERY_START'].dt.tz_convert('Europe/Paris').dt.tz_localize(None)

In [74]:
# Effectuer la jointure
data_info_test = pd.merge(X_test, data_complet_test, on="DELIVERY_START")

In [75]:
X_test.shape

(4942, 10)

In [76]:
data_info_test.shape

(4942, 15)

In [63]:
# Effectuer la jointure
data_info_test = pd.merge(X_test, data_complet_test, on="DELIVERY_START")

In [64]:
data_info_test.shape

(4942, 15)

In [77]:
data_info_test.to_csv('X_test_add.csv', index=False)